# Backpropagation

### Outline
1. Computational Graphs
2. Backpropagation
3. Implementing Karpathy's `micrograd`

## The chain rule

### Differentials review

Recall that to any variable $x$, we can associate a formal object called **the differential of $x$**, denoted $dx$, which represents an infinitesimal change in $x$. (If $x$ is a constant that never varies, then $dx = 0$, indicating that $x$ never changes.) More generally, given a vector $x = (x_1,\dotsc,x_n) \in \mathbb{R}^n$ (which, unless otherwise specified, should always be thought of as a **column vector**), we define the differential of $x$ to be the vector with entry-wise differentials:
\begin{equation*}
    dx = (dx_1,\dotsc,dx_n) \in \mathbb{R}^n.
\end{equation*}
We try to provide some motivation for the rules and concepts of differentials below.

### Gradients review
Recall that the **gradient** of a function $f: \mathbb{R}^n \to \mathbb{R}$ is the **row vector** of partial derivatives:
\begin{equation*}
    \nabla f(x) = \left[ \frac{\partial f}{\partial x_1} \; \frac{\partial f}{\partial x_2} \; \dotsc \; \frac{\partial f}{\partial x_n} \right] \in \mathbb{R}^{1 \times n}.
\end{equation*}
Above, each partial derivative is defined by
\begin{equation*}
    \frac{\partial f}{\partial x_i} = \lim_{h \to 0} \frac{f(x_1,\dotsc,x_i + h,\dotsc,x_n) - f(x_1,\dotsc,x_i,\dotsc,x_n)}{h}.
\end{equation*}
To visually understand this better, recall that the $i$-th coordinate vector is
\begin{equation*}
    e_i = [ 0 \; \dotsb \; 0 \; \underbrace{1}_{i\text{-th}} \; 0 \; \dotsb \; 0 ]^T \in \mathbb{R}^n.
\end{equation*}
These are called "coordinate vectors" because any vector $a = (a_1,\dotsc,a_n)$ can be expressed as the linear combination:
\begin{equation*}
    a = a_1 e_1 + a_2 e_2 + \dotsb + a_n e_n.
\end{equation*}
Now, observe that the $i$-th partial derivative can be expressed as
\begin{equation*}
    \frac{\partial f}{\partial x_i} = \lim_{h \to 0} \frac{f(x + h e_i) - f(x)}{h}.
\end{equation*}
The numerator is the change in $f$ caused by moving a small distance $h$ in the $i$-th direction (more precisely, in the positive direction of the $x_i$-axis). The denominator is the distance moved (i.e. $h$). The ratio then represents the **velocity** of $f$ in the $i$-th direction. So, in the limit as $h$ goes to $0$, the ratio represents the instantaneous velocity of $f$ in the $i$-th direction.

### Directional derivatives

The **directional derivative** of $f$ in the direction of a vector $v = (v_1,\dotsc,v_n) \in \mathbb{R}^n$ is defined as
\begin{equation*}
    \nabla_v f(x) = \lim_{h \to 0} \frac{f(x + h v) - f(x)}{h}.
\end{equation*}
WARNING: Note that $\nabla f$ is a row vector of length $n$ (the length of $x$), whereas $\nabla_v f$ is a scalar (the instantaneous velocity of $f$ at $x$ in the direction of $v$). 

Taking $v$ to be the coordinate vector $e_i$, the directional derivative simply becomes:
\begin{equation*}
    \nabla_{e_i} f(x) = \lim_{h \to 0} \frac{f(x + h e_i) - f(x)}{h} = \frac{\partial f}{\partial x_i}.
\end{equation*}
Note that, in fact we have
\begin{equation*}
    \nabla_{e_i} f(x) = \frac{\partial f}{\partial x_i} = (\nabla f(x)) \cdot e_i,
\end{equation*}
for each coordinate vector $e_1,\dotsc,e_n$. That is, the directional derivative in the direction of $e_i$ is the dot product of $\nabla f$ and $e_i$. 

It turns out (through some careful manipulation of the difference quotient in the definition of the directional derivative) that for any vector $v \in \mathbb{R}^{n\times 1}$, we have the same pattern:
\begin{equation*}
    \nabla_v f(x) = (\nabla f(x)) \cdot v,
\end{equation*}
where $\cdot$ denotes the dot product (or, if you think of $\nabla f$ and $v$ as row and column vectors respectively the usual matrix product). 

**Example.** The one-dimensional case is special, because there are finitely many directions- $2$! (This is because each direction is represented by a unique unit vector.) Taking the positive direction ($v=1$) yields $\nabla_v f(x) = \nabla f(x) = f'(x)$, and taking the negative direction ($v=-1$) yields $\nabla_v f(x) = -\nabla f(x) = -f'(x)$. (Note: the one-dimensional case is also special because the gradient is a scalar!)

### Fundamental property of gradients

Note that
\begin{equation*}
    \nabla_v f(x) = || \nabla f(x) ||  || v ||  \cos(\theta),
\end{equation*}
where $\theta$ is the angle between the direction vector $v$ and the gradient vector $\nabla f(x)$. Thus, if $v$ is a unit vector (representing a direction), then $(\nabla_v f) v$ is the component of $\nabla f(x)$ in the direction of $v$. As $v$ ranges over all directions (unit vectors), we find that
\begin{equation*}
    - || \nabla f(x) || \leqslant \nabla_v f(x) \leqslant || \nabla f(x) ||,
\end{equation*}
where the maximum is achieved when $v$ is pointing in the same direction as the gradient, and the minimum is achieved when $v$ is pointing in the opposite direction.

**MORAL**: The negative gradient $-\nabla f(x)$ points in the direction of steepest descent of $f$ at $x$. This is the direction in which we should move $x$ to decrease $f(x)$ the most. 

### Approximating change in $f$

Let $y = f(x)$. NOTE: $x$ is a vector variable and $y$ is a scalar variable. 

Suppose we start at a point $x \in \mathbb{R}^n$ and move a small distance $h$ in the direction of a unit vector $v \in \mathbb{R}^n$. The original value of $y$ is $f(x)$, and the new value of $y$ is $f(x+hv)$. Thus, the change in $y$ cause by moving from $x$ to $x+ hv$ is 
\begin{equation*}
    \Delta y = f(x+hv) - f(x).
\end{equation*}
On the other hand, by definition, the directional derivative of $f$ in the direction of $v$ is given by
\begin{equation*}
    \nabla_v f(x) = (\nabla f(x)) \cdot v = \lim_{h \to 0} \frac{\Delta y}{h}.
\end{equation*}
When $h$ is very small, then the difference quotient $(\Delta y)/h$ is approximately equal to the directional derivative $\nabla_v f(x)$. Thus, using small $h$, we can approximate the change in $y$ as
\begin{align*}
    \Delta y & \approx  (\nabla_v f(x))h\\
    & = (\nabla f(x) \cdot v) h\\
    & = h \left[ \frac{\partial f}{\partial x_1} \;  \dotsb \; \frac{\partial f}{\partial x_n} \right] \begin{bmatrix} v_1 \\ \vdots \\ v_n \end{bmatrix} \\
    & = \left( \frac{\partial f}{\partial x_1} \right) hv_1 + \dotsb + \left( \frac{\partial f}{\partial x_n} \right) hv_n.
\end{align*}
Note that each $hv_i$ represents how much we change along $x_i$, so it is natural to denote it by $\Delta x_i$. Then, we have
\begin{equation*}
    \Delta y \approx \left( \frac{\partial f}{\partial x_1} \right) \Delta x_1 + \dotsb + \left( \frac{\partial f}{\partial x_n} \right) \Delta x_n.
\end{equation*}
In other words, the change in $y$ when we move from $x$ to $x + (\Delta x_1,\dotsc,\Delta x_n)$ is a weighted average of the individual changes $\Delta x_i$, with the weights equal to the partial derivatives $\frac{\partial f}{\partial x_i}$.

### From approximation to equality: differentials

Differentials are a formal tool that tries to capture the idea of "infinitesimal change". In a nutshell:
- Take every small change $\Delta y$ and $\Delta x_i$ and replace it with an "infinitesimal" change $dy$ and $dx_i$.
- Replace the $\approx$ with an $=$. 

Thus, if $y = f(x) = f(x_1,\dotsc,x_n)$, then we have the differential equation
\begin{align*}
    dy &= (\nabla f(x)) \cdot dx\\
    &= \left[ \frac{\partial f}{\partial x_1} \;  \dotsb \; \frac{\partial f}{\partial x_n} \right] \begin{bmatrix} dx_1 \\ \vdots \\ dx_n \end{bmatrix} \\
    &= \left( \frac{\partial f}{\partial x_1} \right) dx_1 + \dotsb + \left( \frac{\partial f}{\partial x_n} \right) dx_n.
\end{align*}
This is what we referred to as the **Fundamental Rule of Differentials**. Bottom line: if we replace each $dx_i$ with $\Delta x_i$, and the "$=$" sign to a "$\approx$" sign, we get the approximation we had before.

### More general gradients

Suppose we have variable vectors $x = (x_1,\dotsc,x_n) \in \mathbb{R}^n$ and $y = (y_1,\dotsc,y_k) \in \mathbb{R}^k$ such that $y = F(x)$, for some differentiable function 
\begin{equation*}
    F: \mathbb{R}^n \to \mathbb{R}^k.
\end{equation*}
Giving such a function $F$ is equivalent to giving $k$ functions $F_1,\dotsc,F_k : \mathbb{R}^n \to \mathbb{R}$, such that
\begin{equation*}
    y = \begin{bmatrix} y_1 \\ \vdots \\ y_k \end{bmatrix} = \begin{bmatrix} F_1(x) \\ \vdots \\ F_k(x) \end{bmatrix} = F(x) \; \in \mathbb{R}^k.
\end{equation*}
Each equation $y_i = F_i(x)$ gives rise to a differential equation
\begin{equation*}
    dy_i = \left[ \frac{\partial F_i}{\partial x_1} \;  \dotsb \; \frac{\partial F_i}{\partial x_n} \right] \begin{bmatrix} dx_1 \\ \vdots \\ dx_n \end{bmatrix} = \left( \frac{\partial F_i}{\partial x_1} \right) dx_1 + \dotsb + \left( \frac{\partial F_i}{\partial x_n} \right) dx_n.
\end{equation*}
These differential equations can be combined into a single matrix equation:
\begin{equation*}
    \begin{bmatrix} dy_1 \\ \vdots \\ dy_k \end{bmatrix} = \begin{bmatrix} \frac{\partial F_1}{\partial x_1} & \dotsb & \frac{\partial F_1}{\partial x_n} \\ \vdots & \ddots & \vdots \\ \frac{\partial F_k}{\partial x_1} & \dotsb & \frac{\partial F_k}{\partial x_n} \end{bmatrix} \begin{bmatrix} dx_1 \\ \vdots \\ dx_n \end{bmatrix}.
\end{equation*}
The matrix of partial derivatives on the right is called the **Jacobian matrix** of $F$, or simply, the **gradient** of $F$. It is denoted by
\begin{equation*}
    \nabla F(x) = \begin{bmatrix} \nabla F_1(x) \\ \vdots \\ \nabla F_k(x) \end{bmatrix} =  \begin{bmatrix} \frac{\partial F_1}{\partial x_1} & \dotsb & \frac{\partial F_1}{\partial x_n} \\ \vdots & \ddots & \vdots \\ \frac{\partial F_k}{\partial x_1} & \dotsb & \frac{\partial F_k}{\partial x_n} \end{bmatrix} \in \mathbb{R}^{k\times n}.
\end{equation*}

Thus, we have a generalized version of the **Fundamental Rule of Differentials**:
\begin{equation*}
    \textup{If $y = F(x)$, then $dy = \nabla F(x) \cdot dx$.}
\end{equation*}
Note here that:
\begin{align*}
    \textup{Shape of } dy & = \textup{Shape of } y = k \times 1\\
    \textup{Shape of } dx & = \textup{Shape of } x = n \times 1\\
    \textup{Shape of } \nabla F(x) & = k \times n.
\end{align*}
Thus, the differential equation makes sense:
\begin{equation*}
    \underbrace{dy}_{k \times 1} = \underbrace{\nabla F(x)}_{k \times n} \cdot \underbrace{dx}_{n \times 1}.
\end{equation*}

### Chain rule

Suppose we have variable vectors
\begin{align*}
    x & = (x_1,\dotsc,x_n) \in \mathbb{R}^n,\\
    u & = (u_1,\dotsc,u_k) \in \mathbb{R}^k,\\
    y & = (y_1,\dotsc,y_m) \in \mathbb{R}^m,
\end{align*}
which are related by the equations
\begin{equation*}
    u = F(x) \quad \textup{and} \quad y = G(u),
\end{equation*}
for some differentiable functions $F: \mathbb{R}^n \to \mathbb{R}^k$ and $G: \mathbb{R}^k \to \mathbb{R}^m$. Let $H$ be the composition of $F$ and $G$, i.e. $H = G \circ F$. Then, we have
\begin{equation*}
    y = H(x) = G(F(x)) = G(u).
\end{equation*}
We can apply the Fundamental Rule of Differentials to $F$ and $G$:
\begin{align*}
    \underbrace{dy}_{m \times 1} & = \underbrace{\nabla G(u)}_{m \times k} \cdot \underbrace{du}_{k \times 1}\\
    \underbrace{du}_{k \times 1} & = \underbrace{\nabla F(x)}_{k \times n} \cdot \underbrace{dx}_{n \times 1}.
\end{align*}
Substituting the second equation into the first, we get
\begin{equation}\tag{1}
    \underbrace{dy}_{m \times 1}  = \underbrace{\nabla G(u)}_{m \times k} \cdot \underbrace{\nabla F(x)}_{k \times n} \cdot \underbrace{dx}_{n \times 1}.
\end{equation}
On the other hand, we can apply the Fundamental Rule of Differentials to $H$:
\begin{equation}\tag{2}
    \underbrace{dy}_{m \times 1} = \underbrace{\nabla H(x)}_{m \times n} \cdot \underbrace{dx}_{n \times 1}.
\end{equation}
Comparing the right-hand side of (1) and (2), we obtain the following equality of matrices:
\begin{equation*}
    \underbrace{\nabla H(x)}_{m \times n} = \underbrace{\nabla G(u)}_{m \times k} \cdot \underbrace{\nabla F(x)}_{k \times n}.
\end{equation*}
This is the **chain rule** for gradients. In a nutshell, it says that
\begin{equation*}
    \textup{If $H = G \circ F$, \quad then \quad $\nabla H(x) = \nabla G(F(x)) \cdot \nabla F(x)$.}
\end{equation*}

### Example: Entropy of a discrete probability distribution
Suppose we have a discrete probability distribution $p = (p_1,\dotsc,p_k)$, where $p_i \geqslant 0$ and $\sum_{i=1}^n p_i = 1$. That is, $p$ is a point in the probability simplex $\Delta_k$. Recall that the **entropy** of $p$ is defined as
\begin{equation*}
    H(p) = \sum_{i=1}^k p_i \underbrace{(- \log p_i)}_{\textup{surprisal of class $i$}},
\end{equation*}
where, by convention, $0 \log 0 = 0$.

Recall that $H(p)$ is a single number that quantifies the "intrinsic uncertainty" of the distribution $p$. The smallest possible value of $H(p)$ is $H(p) = 0$, which occurs exactly when the distribution is completely deterministic, i.e. $p_i = 1$ for some $i$ and $p_j = 0$ for all $j \neq i$. This makes sense because if one class has probability $1$, then there is no uncertainty whatsover!

On the other hand, the distribution has (intuitively) the most uncertainty when all classes are equally likely, i.e. $p_i = 1/k$ for all $i$. In this case, we have nothing to distinguish the classes, so there is no preference for which class to choose. But... how can one prove this intuitive statement?

Well, one natural strategy is to compute the gradient of $H(p)$ with respect to $p$. Since we are searching for a local max, we can set the gradient to $0$ and solve for $p$. There is a problem with this however: if we choose the first $k-1$ probabilities, then the $k$-th probability is forced to be $p_k = 1 - (p_1 + \dotsb + p_{k-1})$. Thus, $H$ is secretly a function of $k-1$ variables $H(p_1,\dotsc,p_{k-1})$, not of $k$ variables. 

To make this clearer, let $F: \mathbb{R}^{k-1} \to \mathbb{R}^k$ be defined by
\begin{equation*}
    F(x) = F(x_1,\dotsc,x_{k-1}) = \left( \underbrace{x_1}_{F_1(x)},\dotsc,x_{k-1}, \underbrace{1 - \sum_{i=1}^{k-1} x_i}_{F_k(x)} \right) \in \mathbb{R}^k.
\end{equation*}
Let's define $L : \mathbb{R}^k \to \mathbb{R}^k$ to be the function
\begin{equation*}
    L(p) = L(p_1,\dotsc,p_k) = (\underbrace{-p_1 \log p_1}_{L_1(p)}, \dotsc, \underbrace{-p_k \log p_k}_{L_k(p)}) \in \mathbb{R}^k.
\end{equation*}
Finally, let's define  $G: \mathbb{R}^k \to \mathbb{R}$ to be the function
\begin{equation*}
    G(u_1,\dotsc,u_k) = u_1 + \dotsb + u_k \in \mathbb{R}.
\end{equation*}
Then, the entropy function $H(x_1,\dotsc,x_{k-1})$ can be expressed as the composition
\begin{equation*}
    H(x_1,\dotsc,x_{k-1}) = G(L(F(x_1,\dotsc,x_{k-1}))).
\end{equation*}
Visually, we have
\begin{equation*}
    x \xmapsto{F} p \xmapsto{L} u \xmapsto{G} y.
\end{equation*}
Thus, we have differential equations
\begin{align*}
    dy &= \nabla G(u) \cdot du\\
    &= \nabla G(u) \cdot \nabla L(p) \cdot dp\\
    &= \nabla G(u) \cdot \nabla L(p) \cdot \nabla F(x) \cdot dx.
\end{align*}
Thus, by the chain rule, we have
\begin{equation*}
    \underbrace{\nabla H(x)}_{1 \times (k-1)} = \underbrace{\nabla G(u)}_{1 \times k} \cdot \underbrace{\nabla L(p)}_{k \times k} \cdot \underbrace{\nabla F(x)}_{k \times (k-1)}.
\end{equation*}

Now, let's compute gradients for each step, starting from the right. 

First, since $y = G(u)$, we have
\begin{align*}
    \nabla G(u) & = \left[ \frac{\partial G}{\partial u_1} \; \frac{\partial G}{\partial u_2} \; \dotsb \; \frac{\partial G}{\partial u_k} \right] = \left[ 1 \; 1 \; \dotsb \; 1 \right] \in \mathbb{R}^{1 \times k}.
\end{align*}
Next, since $u = L(p)$, where $u,p \in \mathbb{R}^k$, the gradient is a matrix of shape $k \times k$:
\begin{equation*}
    \nabla L(p)  = \begin{bmatrix} \frac{\partial L_1}{\partial p_1} & \frac{\partial L_1}{\partial p_2} & \dotsb & \frac{\partial L_1}{\partial p_k} \\ \vdots & \ddots & \vdots \\ \frac{\partial L_k}{\partial p_1} & \frac{\partial L_k}{\partial p_2} & \dotsb & \frac{\partial L_k}{\partial p_k} \end{bmatrix} = \begin{bmatrix} -\log p_1 - 1 & 0 & \dotsb & 0 \\ 0 & -\log p_2 - 1 & \dotsb & 0 \\ \vdots & \vdots & \ddots & \vdots \\ 0 & 0 & \dotsb & -\log p_k - 1 \end{bmatrix} \in \mathbb{R}^{k \times k}.
\end{equation*}
Finally, consider the equation $p = F(x)$. Since $x$ is of shape $(k-1) \times 1$ and $p$ is of shape $k \times 1$, the gradient is a matrix of shape $k \times (k-1)$:
\begin{align*}
    \nabla F(x)  = \begin{bmatrix} \frac{\partial F_1}{\partial x_1} & \frac{\partial F_1}{\partial x_2} & \dotsb & \frac{\partial F_1}{\partial x_{k-1}} \\ \vdots & \ddots & \vdots \\ \frac{\partial F_k}{\partial x_1} & \frac{\partial F_k}{\partial x_2} & \dotsb & \frac{\partial F_k}{\partial x_{k-1}} \end{bmatrix} = \begin{bmatrix} 
    1       & \dotsb & 0        & 0  \\ 
    \vdots  & \ddots & \vdots   & \vdots \\  
    0       & \dotsb & 0        & 1  \\
    -1      & \dotsb & -1       & -1 \end{bmatrix} \in \mathbb{R}^{k \times (k-1)}.
\end{align*}
Then, the chain rule tell us that
\begin{align*}
    \nabla H(x) &= \nabla G(u) \cdot \nabla L(p) \cdot \nabla F(x)\\
    & = \underbrace{\left[ 1 \; 1 \; \dotsb \; 1 \right]}_{1 \times (k-1)} \cdot
    \underbrace{\begin{bmatrix} -\log p_1 - 1 & 0 & \dotsb & 0 \\ 0 & -\log p_2 - 1 & \dotsb & 0 \\ \vdots & \vdots & \ddots & \vdots \\ 0 & 0 & \dotsb & -\log p_k - 1 \end{bmatrix}}_{k \times k} \cdot
    \underbrace{\begin{bmatrix}
    1       & \dotsb & 0        & 0  \\
    \vdots  & \ddots & \vdots   & \vdots \\
    0       & \dotsb & 0        & 1  \\
    -1      & \dotsb & -1       & -1 \end{bmatrix}}_{k \times (k-1)} \in \mathbb{R}^{1 \times (k-1)}\\
    & = \underbrace{\left[ -(1 + \log p_1) \; \dotsb \; -(1 + \log p_{k-1}) \; -(1 + \log p_k) \right]}_{1 \times k} \underbrace{\begin{bmatrix}
    1       & \dotsb & 0        & 0  \\
    \vdots  & \ddots & \vdots   & \vdots \\
    0       & \dotsb & 0        & 1  \\
    -1      & \dotsb & -1       & -1 \end{bmatrix}}_{k \times (k-1)}\\
    & = \left[ \big( -1 - \log p_1 + 1 + \log p_k \big) \; \dotsb \; \big( -1 - \log p_{k-1} + 1 + \log p_k \big) \right] \in \mathbb{R}^{1 \times (k-1)}\\
    & = \left[ \log \left(\frac{p_k}{p_1} \right) \; \dotsb \; \log \left(\frac{p_k}{p_{k-1}} \right) \right] \in \mathbb{R}^{1 \times (k-1)}.
\end{align*}
It follows that $\nabla H(x) = 0$ if and only if
\begin{equation*}
    \log(p_k) = \log(p_1) = \dotsb = \log(p_{k-1}),
\end{equation*}
which is equivalent to $p_1 = \dotsb = p_k$. Since these add up to $1$, we conclude that $\nabla H(x_1,\dotsc,x_{k-1}) = \nabla H(p_1,\dotsc,p_{k-1})$ is $0$ if and only if $p_1 = \dotsb = p_k = 1/k$. As expected, this is the point where the entropy function $H(p_1,\dotsc,p_{k-1})$ is maximized!

**Remark.** The above argument is incomplete. Indeed, the gradient being $0$ at a point does not imply that the point is a local max or min. For this, we need to investigate the curvature (concavity) of $H$ at the point, which means we need to compute the *Hessian* of $H$, which is the matrix of second-order partial derivatives. In this case, it turns out to be a diagonal matrix with diagonal entries $-1/{p_i}$. This matrix is negative definite at the point $p = (1/k,\dotsc,1/k)$, so we conclude that $H(p)$ is concave down at this point. Thus, the point $p_1 = \dotsb = p_k = 1/k$ is indeed a local max of $H(p)$.